In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
#from IPython.display import Image
import numpy as np

## Scalar initial value problem (Example 10.9 in Smith)


$$\frac{du}{dt} = -a(\omega) u$$

$$u(t=0,\omega) = b$$

$$a \sim N(a_0,\sigma_a^2)$$

The damping rate $a$ is random with $a_0 = 1$, $\sigma_a = 0.25$. The initial condition is fixed and deterministic $b=b_0 = 10$. 

In [ ]:
a0 = 1
sigma_a = 0.25
b0 = 10

The analytical solution is $$u(t) = b e^{-at}$$

In [ ]:
def u(b,a,t): #Exact solution
    return b*np.exp(-a*t)

$$E[u(t)] = b e^{-a_0 t} e^{\sigma_a^2 t^2/2}$$

$$var[u(t)] = e^{-2a_0 t} b^2 (e^{2 \sigma_a^2 t^2} - e^{\sigma_a^2 t^2})$$

In [ ]:
nt=100
t = np.linspace(0, 12, nt)
t2 = t.reshape(nt,1)
umean_exact = b0*np.exp(-a0*t)*np.exp(sigma_a**2*t**2/2)
uvar_exact = b0**2*np.exp(-2*a0*t)*(np.exp(2*sigma_a**2*t**2)-np.exp(sigma_a**2*t**2))

In [ ]:
plt.plot(t,umean_exact,'k--',label='exact')
plt.plot(t,umean_exact+2*np.sqrt(uvar_exact),'k--')
plt.plot(t,umean_exact-2*np.sqrt(uvar_exact),'k--')
plt.xlabel('Time (s)'),plt.ylabel('Displacement (m)')
plt.legend(['Mean'])
plt.title('2-$\sigma$ credible intervals')
plt.legend(loc="upper right")

## Direct simulation

Sample damping rates and compute trajectories.

In [ ]:
plt.plot(t,u(b0,np.random.normal(a0,sigma_a,1000),t2));
plt.xlabel('Time (s)'),plt.ylabel('Displacement (m)')

Compute statistics (mean and starndard deviation) of the resulting trajectories.

In [ ]:
Nsamples = 100000
udirect = u(b0,np.random.normal(a0,sigma_a,Nsamples),t2)
umean = np.mean(udirect,axis=1)
uplus = umean + 2*np.std(udirect,axis=1)
uminus = umean - 2*np.std(udirect,axis=1)
fig, ax  = plt.subplots()
ax.plot(t,umean,label='simulation')
ax.plot(t,uplus)
ax.plot(t,uminus)
ax.plot(t,umean_exact,'k--',label='exact')
ax.plot(t,umean_exact+2*np.sqrt(uvar_exact),'k--')
ax.plot(t,umean_exact-2*np.sqrt(uvar_exact),'k--')
plt.xlabel('Time (s)'),plt.ylabel('Displacement (m)')
plt.legend(['Mean'])
plt.title('2-$\sigma$ credible intervals')
plt.legend(loc="upper right")

Note the intervals grow and become unbounded for large $t$ because some of the damping rates are negative.

## Stochastic Spectral

We seek approximate solutions $$u^K(t,Q) = \sum_{k=0}^K u_k(t) \psi_k(Q)$$
Subject to $$\left\langle \frac{du^K}{dt} + a^N u^K,\psi_i \right\rangle_\rho = 0\,, \qquad i=0,\dots, K$$
Or
$$\left\langle \frac{du^K}{dt},\psi_i \right\rangle_\rho = \left\langle_\rho a^N u^K,\psi_i \right\rangle_\rho \,, \qquad i=0,\dots, K$$
Or
$$ \int \sum_{k=0}^K \frac{d u_k}{dt} (t) \psi_k(q) \psi_i(q) \rho_Q(q) dq = \int a^N(q) \sum_{k=0}^K u_k(t) \psi_k(q) \psi_i(q) \rho_Q(q) dq$$
$$ a^N(q) = \sum_{n=0}^N a_n \psi_n(q) = a_0 + \sigma_a q$$

This yields $K+1$ differential equations $$ \frac{du_i}{dt} = \frac{1}{\gamma_i} \sum_{n=0}^N \sum_{k=0}^K a_n u_k(t) e_{ink} $$
where $\gamma_i = E[\psi_i^2]$ and $e_{ink} = E[ \psi_i \psi_n \psi_k]$.

Or
$$ \frac{d\mathbf{u}}{dt}= \mathbf{A} \mathbf{u} $$

# <span style="color:red"> Choose $K$</span>
Try $K=6, 8, 12$.

In [ ]:
K=6

In [ ]:
def e_ink(i,n,k):
    s2 = i + n + k
    s = (i + n + k)/2
    if np.mod(s2,2)==1:
        f = 0
    elif ((s<i) | (s<n) | (s<k)):
        f = 0
    else:
        f = np.math.factorial(i)*np.math.factorial(n)*np.math.factorial(k)/np.math.factorial(s-i)/np.math.factorial(s-n)/np.math.factorial(s-k)
    return f

In [ ]:
A = np.zeros(shape=(K+1,K+1))
gamma = np.zeros(K+1)
for i in range(K+1):
    gamma[i] = np.math.factorial(i)
    for k in range(K+1):
        A[i,k] = -1/gamma[i]*(a0*e_ink(i,0,k)+sigma_a*e_ink(i,1,k))

In [ ]:
plt.pcolor(A)
plt.colorbar()

In [ ]:
from scipy.integrate import odeint
def dU_dt(U, t, A):
    # Here U is a vector such that y=U[0] and z=U[1]. This function should return [y', z']
    return A.dot(U)

In [ ]:
U0 = np.zeros(K+1)
U0[0] = b0
UK = odeint(dU_dt, U0, t, args=(A,))
UKmean = UK[:,0]
UKvar = np.sum(gamma[1:]*UK[:,1:]**2,axis=1)

In [ ]:
fig, ax  = plt.subplots()
ax.plot(t,UKmean)
ax.plot(t,UKmean + 2*np.sqrt(UKvar))
ax.plot(t,UKmean - 2*np.sqrt(UKvar))
ax.plot(t,umean_exact,'k--')
ax.plot(t,umean_exact+2*np.sqrt(uvar_exact),'k--')
ax.plot(t,umean_exact-2*np.sqrt(uvar_exact),'k--')

## Discrete projection
Also called pseudospectral.
$$ u_k(t) = \frac{1}{\gamma_k} \langle(u(t,q),\psi_k \rangle = \frac{1}{\gamma_i} \int u(t,q)\psi_k(q) \rho_Q(q) dq \approx \frac{1}{\gamma_i} \sum_{r=1}^R u(t,q^r)\psi_k(q^r) \rho_Q(q^r) w^r$$
Note requires solving for $u(t,q^r)$. Non-intrusive.

In [ ]:
from numpy.polynomial import HermiteE as H

In [ ]:
qq = np.linspace(-2, 2, 100)
for i in range(4): ax = plt.plot(qq, H.basis(i)(qq), lw=2, label="$H_%d$"%i)
plt.legend(loc="lower left")

$$u_k(t) = \frac{1}{\gamma_k} \sum_{r=1}^R u(t,q^r) \psi_k(q^r) \rho_Q(q^r) w^r$$
Use Gauss-Hermite quadrature points. Check normalization. They come in different flavors.

In [ ]:
R = 16
q,w = np.polynomial.hermite_e.hermegauss(R)
w = w/np.sqrt(2*np.pi)
np.sum(w)

$$u_k(t) = \frac{1}{\gamma_i} \sum_{r=1}^R u(t,q^r)\psi_k(q^r)  w^r$$

In [ ]:
UKp = np.zeros(shape=(nt,K+1))
for k in range(K+1):
    UKp[:,k] = np.sum(H.basis(k)(q)*w*u(b0,a0+sigma_a*q,t2),axis=1)/gamma[k]

In [ ]:
UKpmean = UKp[:,0]
UKpvar = np.sum(gamma[1:]*UKp[:,1:]**2,axis=1)

In [ ]:
fig, ax  = plt.subplots()
ax.plot(t,UKpmean)
ax.plot(t,UKpmean + 2*np.sqrt(UKpvar))
ax.plot(t,UKpmean - 2*np.sqrt(UKpvar))
ax.plot(t,umean_exact,'k--')
ax.plot(t,umean_exact+2*np.sqrt(uvar_exact),'k--')
ax.plot(t,umean_exact-2*np.sqrt(uvar_exact),'k--')

## Collocation
Find coefficients $u_k(t)$ that make $u(t,q^m) \approx u^K(t,q^m)$, $m=1,\dots,M$. $q^m$ are collocation points. 

Least-squares problem.

$$  u^K(t,q^m) = \sum_{k=0}^K u_k(t) \psi_k(q^m) = u(t,q^m)\,, m = 1,\dots,M$$
Or
$$ 
\begin{bmatrix} 
\psi_0(q^1) & \cdots & \psi_k(q^1) \\
\vdots & & \vdots\\
\psi_0(q^M) & \cdots & \psi_K(q^M)
\end{bmatrix}
\begin{bmatrix} 
u_0(t)\\
\vdots\\
u_K(t)
\end{bmatrix} 
=\begin{bmatrix} 
u(t,q^1)\\
\vdots\\
u(t,q^M)
\end{bmatrix} 
$$

Note rhs requires $M$ solutions, comparable to discrete projection.

Let's just use the Gauss-Hermite points. (Scaling seems to work better. What is a good choice?)

In [ ]:
q = q/1.2
rhs = u(b0,a0+sigma_a*q,t2)
rhs = np.swapaxes(rhs,0,1)
rhs.shape

In [ ]:
from numpy.polynomial.hermite_e import hermevander
from numpy.polynomial.hermite_e import hermefit

In [ ]:
UKc = hermefit(q,rhs,K)
UKc = np.swapaxes(UKc,0,1)
UKc.shape

In [ ]:
UKcmean = UKc[:,0]
UKcvar = np.sum(gamma[1:]*UKc[:,1:]**2,axis=1)

In [ ]:
fig, ax  = plt.subplots()
ax.plot(t,UKcmean)
ax.plot(t,UKcmean + 2*np.sqrt(UKcvar))
ax.plot(t,UKcmean - 2*np.sqrt(UKcvar))
ax.plot(t,umean_exact,'k--')
ax.plot(t,umean_exact+2*np.sqrt(uvar_exact),'k--')
ax.plot(t,umean_exact-2*np.sqrt(uvar_exact),'k--')